In [1]:
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
import glob
import os
pd.options.mode.chained_assignment = None

### This script is using all the UK_2012.pkl data and transfer to 
UK_Oxford_2012.pkl data and then transfer to
1. UK_Oxford_skill_processed_all.pkl
2. UK_Oxford_skillClluster_processed_all.pkl

In [2]:
def saveOxfordDataByYear(year):
    country = 'UK'
    df = pd.read_pickle(country + '_' + str(year) + '.pkl')
    df = df[(df['CanonCounty'] == 'Oxfordshire')]
    df.to_pickle(country + '_' + 'Oxford' + '_' + str(year) + '.pkl')

In [ ]:
## extract the oxford data
for year in range(2013,2020):
    saveOxfordDataByYear(year)

### UK_Oxford_skill_processed_all.pkl

In [8]:
### expand the data by skills
def expand_skill(df):
    s = df.apply(lambda x: pd.Series(x['CanonSkills']), axis=1).stack(
    ).reset_index(level=1, drop=True) # takes a lot of time 
    s.name = 'CanonSkills'
    s = df.drop('CanonSkills', axis=1).join(s)
   
    return s


In [11]:
## save all the expanded data into myDataFrame
myDataFrame = pd.DataFrame()
for year in range(2012,2020):
    
    df = pd.read_pickle('UK_Oxford' + '_' + str(year) + '.pkl') # read every year data
    df2 = expand_skill(df) # process every year data
    myDataFrame = myDataFrame.append(df2)


In [14]:
## save processed skill oxford data 
myDataFrame.to_pickle(country + '_' + 'Oxford' + '_skill_processed_all'  + '.pkl')
                     

### UK_Oxford_skillCluster_processed_all.pkl

In [4]:
### expand the data by skillsClusters
def element_to_list(element):
    return str(element).replace(";", "&").replace('"', "").split("&")

def expand_skillCluster(df):
    df = df[['JobID', 'SICCode','JobDate','CanonCity','CanonCounty',
             'LocalAuthorityDistrict',
             'CanonSkillClusters', 'SOCCode', 'CanonSkills']]
    df['date'] = pd.to_datetime(df['JobDate'])
    
    df['CanonSkillClusters'] = df['CanonSkillClusters'].apply(element_to_list)
    
    s = df.apply(lambda x: pd.Series(x['CanonSkillClusters']), axis=1).stack(
    ).reset_index(level=1, drop=True) # takes a lot of time 
    s.name = 'CanonSkillClusters'
    s = df.drop('CanonSkillClusters', axis=1).join(s)
   
    return s

In [6]:
## save all the expanded data into myDataFrame
myDataFrame = pd.DataFrame()
for year in range(2012,2020):
    
    df = pd.read_pickle('UK_Oxford' + '_' + str(year) + '.pkl') # read every year data
    df2 = expand_skillCluster(df) # process every year data
    myDataFrame = myDataFrame.append(df2)

In [8]:
## save processed skill cluster oxford data 
myDataFrame.to_pickle('UK' + '_' + 'Oxford' + '_skillCluster_processed_all'  + '.pkl')
   